Load and process the data



In [2]:
! pip install transformers
import torch
import json
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32
learning_rate = 2e-5
max_len = 180
batch_size = 32

epochs = 4

# dev = xm.xla_device()


dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))

class DataGetter:
    def __init__(self, path):
        self.path = path
        self.sentence = []
        self.label = []
        self.tokenized_sentence = []
        self.numeric_label = []
        self.tag2idx = {}

    def bio_converter(self):
        # Converting a normal font style file to BIO sequence labeling style
        with open(self.path, 'r') as fp:
            for line in fp.readlines():
                sentence = json.loads(line)  # read the json file line by line
                labels = sentence['label']
                text = sentence['text']
                text_list = list(text)
                target = []  # the 'target' list stores the output
                for i in range(len(text)):  # filling the output list with 'O' tag, which is the default tag
                    target.append('O')
                for key in labels:
                    entity_type = key
                    entity_dict = labels[key]  # e.g. {'叶老桂': [[9, 11]]}
                    for entity_name in entity_dict:
                        # searching for the label indexes in the target,
                        # replacing them with appropriate tag, the start tag is marked as 'B',
                        # 'I' tag is marked until the end of the label
                        entity_start_index = entity_dict[entity_name][0][0]
                        entity_end_index = entity_dict[entity_name][0][1]
                        entity_length = entity_end_index - entity_start_index + 1
                        target[entity_start_index] = 'B-' + str(entity_type)
                        if entity_length != 1:
                            for i in range(entity_start_index + 1, entity_end_index + 1):
                                target[i] = 'I-' + str(entity_type)
                self.label.append(target)
                self.sentence.append(text_list)
        fp.close()
        print("数据保存完毕")
        self.convert_labels_to_id()
        return self.label, self.sentence

    def convert_labels_to_id(self):
        self.count_labels()
        self.numeric_label = [[self.tag2idx.get(l) for l in lab] for lab in self.label]

    def count_labels(self):
      # 通过遍历整个label数组得到该文件中一共有多少种label组合。其中Label组合是指形如i-geo 或者 b-per这类的BIO标注与tag的组合
        tag_values = []
        for label_list in self.label:
            for labels in label_list:
                tag_values.append(labels)
        tag_values = list(set(tag_values))
        self.tag2idx = {t: i for i, t in enumerate(tag_values)}

    def get_tag2idx(self):
      # 得到储存有所有tag与index的字典tag2idx
      return self.tag2idx

    def get_numeric_labels(self):
      return self.numeric_label
    

     |████████████████████████████████| 778kB 4.6MB/s 
     |████████████████████████████████| 3.0MB 23.0MB/s 
     |████████████████████████████████| 890kB 42.0MB/s 
     |████████████████████████████████| 1.1MB 55.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e01f6eb0edc5278ca8395e5c97982a0a745e9a28887a9eca6923745d7aa44980
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Tesla T4


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
train_data_loader = DataGetter('/content/train.json')
train_labels, train_sentences = train_data_loader.bio_converter()
tag2idx = train_data_loader.get_tag2idx()
train_labels = train_data_loader.get_numeric_labels()
print(tag2idx) # "O"'s  id = 3

数据保存完毕
{'B-movie': 0, 'I-movie': 1, 'I-scene': 2, 'I-book': 3, 'B-position': 4, 'B-book': 5, 'B-game': 6, 'I-organization': 7, 'B-scene': 8, 'I-company': 9, 'I-game': 10, 'B-government': 11, 'B-company': 12, 'O': 13, 'I-name': 14, 'B-organization': 15, 'I-position': 16, 'I-address': 17, 'I-government': 18, 'B-address': 19, 'B-name': 20}


Custom our data set to meet Bert's requirement

In [7]:
class CustomData:
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.labels = labels
        self.max_len = max_len
        self.len = len(sentences)

    def __getitem__(self, index):
      # 当bert解压数据集时会调用函数getitem。在这里我们override getitem函数使得当bert解压data loader时返回三个tensor字典
        sentence = self.sentences[index]
        label = self.labels[index]
        inputs = tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        label.extend([13] * max_len)  # 当我们对数据进行padding时我们把多余的项用标记为O的无关项填充
        label = label[:max_len]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'tags': torch.tensor(label, dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [8]:
training_set = CustomData(tokenizer, train_sentences, train_labels, max_len)
train_params = {'batch_size': batch_size,
                'shuffle': True,
                'num_workers': 0
                }
#将training的参数以字典形式储存
training_loader = DataLoader(training_set, **train_params) # 打包成tensor的dataloader，其中包括data和训练用的参数

Set up the bert model


In [9]:
from transformers import BertForTokenClassification, AdamW, BertModel, BertConfig
import torch

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        super().__init__()
        self.l1 = BertForTokenClassification.from_pretrained(
            'bert-base-chinese',
            num_labels=len(tag2idx),
        )

    def forward(self, ids, masks, labels):
        output = self.l1(ids, masks, labels=labels)
        return output

model = BERTClass()
model.to(dev)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-c

BERTClass(
  (l1): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(21128, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
       

Set up for fine tune

In [15]:
from transformers import get_linear_schedule_with_warmup

optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate) # set up the optimizer

max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(training_loader) * epochs
print('total steps: %d'%total_steps)
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

total steps: 1344


Fit the model for NER

In [11]:
def train(epoch):
    model.train()
    for step, data in enumerate(training_loader, 0):
        ids = data['ids'].to(dev, dtype=torch.long)
        mask = data['mask'].to(dev, dtype=torch.long)
        targets = data['tags'].to(dev, dtype=torch.long)

        loss = model(ids, mask, labels=targets)[0]

        if step%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        optimizer.step()
        scheduler.step()

In [12]:
for epoch in range(4):
  train(epoch)

Epoch: 0, Loss:  3.041417360305786
Epoch: 1, Loss:  0.15226371586322784
Epoch: 2, Loss:  0.14114871621131897
Epoch: 3, Loss:  0.11926872283220291


Validate the model